## Robo 2DOF

```TODO```

In [13]:
## sympy modulo para trabalhar com equações simbólicas
from sympy import *
import numpy as np

In [14]:
# variaveis
theta1 = Function('theta_1'); theta2 = Function('theta_2');
t,m1,m2,L1,L2, g = symbols('t m1 m2 L_1 L_2 g')

In [15]:
def tf_h(theta, Q):
    tf_matrix = Matrix([[cos(theta),   sin(theta),  0,      Q[0]],
                        [sin(theta),   cos(theta),  0,      Q[1]],
                        [0,            0,           1,      Q[2]],
                        [0,            0,           0,      1]])
    return tf_matrix

In [16]:
P1 = Matrix([[0, L1, 0, 1]])
P1.T

Matrix([
[  0],
[L_1],
[  0],
[  1]])

In [17]:
link1 = tf_h(theta1(t), [0,0,0])*P1.T
link1

Matrix([
[L_1*sin(theta_1(t))],
[L_1*cos(theta_1(t))],
[                  0],
[                  1]])

In [18]:
P2 = Matrix([[0, L2, 0, 1]])
P2.T

Matrix([
[  0],
[L_2],
[  0],
[  1]])

In [52]:
link2 = tf_h(theta2(t)+theta1(t), [link1[0],link1[1],link1[2]])*P2.T
link2

Matrix([
[L_1*sin(theta_1(t)) + L_2*sin(theta_1(t) + theta_2(t))],
[L_1*cos(theta_1(t)) + L_2*cos(theta_1(t) + theta_2(t))],
[                                                     0],
[                                                     1]])

In [53]:
dlk1 = diff(link1[:3,:], t)
dlk1

Matrix([
[ L_1*cos(theta_1(t))*Derivative(theta_1(t), t)],
[-L_1*sin(theta_1(t))*Derivative(theta_1(t), t)],
[                                             0]])

In [54]:
dlk2 = diff(link2[:3,:], t)
dlk2

Matrix([
[ L_1*cos(theta_1(t))*Derivative(theta_1(t), t) + L_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_1(t) + theta_2(t))],
[-L_1*sin(theta_1(t))*Derivative(theta_1(t), t) - L_2*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_1(t) + theta_2(t))],
[                                                                                                                                        0]])

In [86]:
K1 = (1/2*m1*dlk1.T*dlk1)[0]
K1 = simplify(K1)
K1

0.5*L_1**2*m1*Derivative(theta_1(t), t)**2

In [87]:
K2 = (1/2*m2*dlk2.T*dlk2)[0]
K2 = simplify(K2)
K2

0.5*m2*(L_1**2*Derivative(theta_1(t), t)**2 + 2*L_1*L_2*cos(theta_2(t))*Derivative(theta_1(t), t)**2 + 2*L_1*L_2*cos(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + L_2**2*Derivative(theta_1(t), t)**2 + 2*L_2**2*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + L_2**2*Derivative(theta_2(t), t)**2)

In [91]:
V = (m1+m2)*g*link1[1] + m2*g*link2[1]
V

L_1*g*m1*cos(theta_1(t)) + g*m2*(L_1*cos(theta_1(t)) + L_2*cos(theta_1(t) + theta_2(t)))

In [93]:
L = (K1+K2) + V
L

0.5*L_1**2*m1*Derivative(theta_1(t), t)**2 + L_1*g*m1*cos(theta_1(t)) + g*m2*(L_1*cos(theta_1(t)) + L_2*cos(theta_1(t) + theta_2(t))) + 0.5*m2*(L_1**2*Derivative(theta_1(t), t)**2 + 2*L_1*L_2*cos(theta_2(t))*Derivative(theta_1(t), t)**2 + 2*L_1*L_2*cos(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + L_2**2*Derivative(theta_1(t), t)**2 + 2*L_2**2*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) + L_2**2*Derivative(theta_2(t), t)**2)

In [114]:
dtheta1 = diff(theta1(t), t)
ddtheta1 = diff(diff(theta1(t), t),t)
dtheta2 = diff(theta2(t), t)
ddtheta2 = diff(diff(theta2(t), t),t)

collect(simplify(diff(diff(L,dtheta1),t) - diff(L,theta1(t))), [ddtheta1, ddtheta2])

1.0*L_1**2*m1*Derivative(theta_1(t), (t, 2)) + L_1*g*m1*sin(theta_1(t)) + g*m2*(L_1*sin(theta_1(t)) + L_2*sin(theta_1(t) + theta_2(t))) + m2*(-2*L_1*L_2*sin(theta_2(t))*Derivative(theta_1(t), t)*Derivative(theta_2(t), t) - L_1*L_2*sin(theta_2(t))*Derivative(theta_2(t), t)**2 + (L_1*L_2*cos(theta_2(t)) + L_2**2)*Derivative(theta_2(t), (t, 2)) + (L_1**2 + 2*L_1*L_2*cos(theta_2(t)) + L_2**2)*Derivative(theta_1(t), (t, 2)))

In [112]:
M11 = (m1+m2)*L1**2 + L2**2 * m2 +2*L1*L2*m2*cos(theta2(t))
M11 

L_1**2*(m1 + m2) + 2*L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2

In [127]:
M12 = L1*L2*m2*cos(theta2(t)) + L2**2*m2
M12

L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2

In [128]:
collect(simplify(diff(diff(L,dtheta2),t) - diff(L,theta2(t))), [ddtheta1, ddtheta2])

L_2*m2*(L_1*sin(theta_2(t))*Derivative(theta_1(t), t)**2 + L_2*Derivative(theta_2(t), (t, 2)) + g*sin(theta_1(t) + theta_2(t)) + (L_1*cos(theta_2(t)) + L_2)*Derivative(theta_1(t), (t, 2)))

In [130]:
M21 = L2*L1*m2*cos(theta2(t)) + L2**2*m2
M21

L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2

In [131]:
M21 = L2*L1*m2*cos(theta2(t)) + L2**2*m2
M21

L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2

In [132]:
M22 = L2**2*m2
M22

L_2**2*m2

In [135]:
M = Matrix([[M11,M12],[M21, M22]])
M

Matrix([
[L_1**2*(m1 + m2) + 2*L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2, L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2],
[                     L_1*L_2*m2*cos(theta_2(t)) + L_2**2*m2,                              L_2**2*m2]])

In [12]:
simplify(diff(L1*cos(theta1(t))+ L2*cos(theta1(t)+ theta2(t)), theta1(t)) + diff(L1*cos(theta1(t))+ L2*cos(theta1(t)+ theta2(t)), theta2(t)))

-L_1*sin(theta_1(t)) - 2*L_2*sin(theta_1(t) + theta_2(t))